In [11]:
import torch
from torchvision import transforms
from PIL import Image
import torch.nn as nn

# Define the CNN model architecture to match the saved model
class CNNModel(nn.Module):
    def __init__(self, num_classes=7):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)  # 16 output channels
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)  # 32 output channels
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)  # 64 output channels
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(64 * 28 * 28, 128)  # 128 output features
        self.fc2 = nn.Linear(128, num_classes)  # 7 output classes
        self.dropout = nn.Dropout(0.5)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.pool(self.relu(self.conv3(x)))
        x = x.view(-1, 64 * 28 * 28)  # Flatten the tensor
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# Load the CNN model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_path = "cnn_waste_model.pth"

model = CNNModel(num_classes=7)
model.load_state_dict(torch.load(model_path, map_location=device))
model.to(device)
model.eval()

categories = ["Cardboard", "E-Waste", "Glass", "Medical", "Metal", "Paper", "Plastic"]

# Preprocess the image
def preprocess_image(image):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),  # Resize to the input size expected by the CNN
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    return transform(image).unsqueeze(0)

# Classify the image
def classify_image(image_path):
    try:
        image = Image.open(image_path)
        input_tensor = preprocess_image(image).to(device)
        with torch.no_grad():
            output = model(input_tensor)
        _, predicted = torch.max(output, 1)
        confidence = torch.nn.functional.softmax(output, dim=1)[0][predicted].item() * 100
        return categories[predicted], confidence
    except Exception as e:
        print(f"Error classifying image: {e}")
        return "Unknown", 0.0

# Example usage
#image_path = 'C:\\Users\\ksbas\\Desktop\\hproject\\Main_SMW\\plastic_bottle_1.jpeg'
image_path = 'C:\\Users\\ksbas\\Desktop\\hproject\\Main_SMW\\steel-bottle-1.jpg'
category, confidence = classify_image(image_path)
print(f"Classification Result: {category} with {confidence:.2f}% confidence")

Classification Result: Glass with 98.95% confidence
